In [ ]:
PATH_TO_DATA = 'input/hotel/Hotel_Reviews.csv'

# some necessary imports
import os
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
# Utility
import re
from collections import Counter
import logging
import time
import pickle
import itertools
nltk.download('stopwords')

# DATASET
DATASET_ENCODING = "ISO-8859-1"

# TEXT CLENAING
TEXT_CLEANING_RE = "[^A-Za-z0-9]+"

stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

train_df = pd.read_csv(PATH_TO_DATA, encoding =DATASET_ENCODING)

def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

df = train_df[['Negative_Review', 'Positive_Review']]

df.Negative_Review = df.Negative_Review.apply(lambda x: preprocess(x))
df.Positive_Review = df.Positive_Review.apply(lambda x: preprocess(x))

texts = []
labels = []
texts_test = []
labels_test = []
for i in range (60000):
    if df.Negative_Review[i+10000] != "negative":
        texts.append(df.Negative_Review[i+10000])
        labels.append(0)
for i in range (60000):
    if df.Positive_Review[i+10000] != "positive":
        texts.append(df.Positive_Review[i+10000])
        labels.append(1)
for i in range (6000):
    if df.Negative_Review[i] != "negative":
        texts_test.append(df.Negative_Review[i])
        labels_test.append(0)
for i in range (6000):
    if df.Positive_Review[i] != "positive":
        texts_test.append(df.Positive_Review[i])
        labels_test.append(1)
        
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 20  # We will cut reviews after 20 words
max_words = 5000  # We will only consider the top 5,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)  # https://keras.io/preprocessing/text/
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)  # This turns strings into lists of integer indices.

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

d = 0
for i in range (len(sequences)):
    if len(sequences[i-d]) == 0:
        del sequences[i-d]
        del labels[i-d]
        d = d+1
        
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

onehot = np.zeros((data.shape[0], max_words))
for i in range(data.shape[0]):
    for j in range(maxlen):
        temp2 = data [i]
        if temp2[j] != 0:
            a = temp2[j] - 1
            onehot[i][a] = 1
            
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import confusion_matrix
from models import SupervisedDBNClassification


# Loading dataset
X, Y = onehot, labels

# Data scaling
X = (X).astype(np.float32)

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=2/9, random_state=0)

In [ ]:
# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[16, 16],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=10,
                                         batch_size=64,
                                         activation_function='relu',
                                         dropout_p=0.05)
classifier.fit(X_train, Y_train)

# Test
Y_pred = classifier.predict(X_test)
print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))